---
title: Advent of Code 2020
description: a verbose solve of AOC 2020
tags:
- python
date: 2020-12-01
lastmod: 2020-12-01
toc: true
---

The following is a write up of how to solve and the stuff I learved while doing so, thus some of the code is spelled out and slow! So look at the [AOC reddit site](https://www.reddit.com/r/adventofcode/) for ninja level solutions.

I am trying to

- code in a readable fashion, avoiding shortcuts (easier said then done!)
- make visuals as many problems as possible.
- not [mangle up jupyter notebooks too badly](https://docs.google.com/presentation/d/1n2RlMdmv1p25Xy5thJUhkKGvjtV-dkAIsUXP-AL4ffI/edit?usp=sharing). Real code is written using a text editor! Unless you are [Netflix](https://netflixtechblog.com/notebook-innovation-591ee3221233), or course, or using [rmarkdown](https://yihui.org/en/2018/09/xfun-pkg-attach/).

First up, I'm importing all the libs I'll use up here:

In [2]:
# python essentials
import os
import re
import hashlib
import math
from pathlib import Path
from typing import List
from collections import defaultdict, namedtuple, Counter
import itertools

# useful external libs
#import numpy as np
#import pandas as pd

# misc utils
#import requests
#from tqdm.notebook import trange, tqdm # progress bars for slow funcs
#from functools import reduce 

# for plots, cause visuals
import matplotlib.pyplot as plt # goto python viz lib
#import seaborn as sns # prettify matplotlib
from IPython.display import display, Markdown

# javascript plotting for interactive graphs
#import altair as alt
#import plotly.express as px

Some helper functions to avoid rewriting the same code for all the problems:

In [25]:
input_path = Path("data") / "advent-of-code-2020"
github_url = url = "https://github.com/khalido/adventofcode/raw/master/inputs"

def get_input(day:int=1, splitlines=True):
    """takes in the day, year and date, returns the input"""
    
    try:    # load from local disk
        with open(input_path / f"{day}.txt") as f:
            data = f.read().strip()
    except: # else load from github repo
        print(f"Failed to load {year}/{day}.txt from disk")

    if splitlines: data = data.splitlines()

    return data 

def printmd(txt="## testing"):
    display(Markdown(txt))

## Day 1: Report Repair

[#](https://adventofcode.com/2020/day/1) We have a list of numbers and need to find the two numbers that add up to 2020, and multiple them to get the answer.

In [30]:
test1 = """1721
979
366
299
675
1456""".splitlines()

inp1 = get_input(1)

def parse_1(inp): 
    return [int(i) for i in inp]

def solve_1(inp, numbers=2):
    # get the first nums which add up to 2020
    nums = [i for i in itertools.combinations(parse_1(inp), numbers) if sum(i)==2020][0]
    ans = math.prod(nums)
    return ans

assert solve_1(test1) == 514579
printmd(f"Day 1A: **{solve_1(inp1)}**")

Day 1A: **878724**

**Part 2:** Now we want to find the three numbers which sum to 2020 and return the multiple.

Since I made a general func to solve for 1, this is just rerunning the above with numbers set to 3.

In [31]:
assert solve_1(test1, 3) == 241861950
printmd(f"Day 1B: **{solve_1(inp1, 3)}**")

Day 1B: **201251610**

## Day 2: Password policy

[#](https://adventofcode.com/2020/day/2). We have a list of passwords in the form: `password policy: password`, where the first two numbers give a range of acceptable repeats for a character. Each policy has `"lo", "hi", "letter", "password"` which can be saved directly as a namedtuple.

In [22]:
test2 = """1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc""".splitlines()

inp2 = get_input(2)

Policy = namedtuple("Policy", ["lo", "hi", "letter", "password"])

def parse_2(inp):
    data = []

    for pol in [pol.split() for pol in inp]:
        lo, hi = map(int, pol[0].split("-"))
        letter = pol[1].strip(":")
        password = pol[2]
        data.append(Policy(lo, hi, letter, password))
    
    return data

parse_2(test2) # eyeballing to see it works:

[Policy(lo=1, hi=3, letter='a', password='abcde'),
 Policy(lo=1, hi=3, letter='b', password='cdefg'),
 Policy(lo=2, hi=9, letter='c', password='ccccccccc')]

To solve this is straightforward, we just make sure the count of the given char is within the acceptable range, which python makes easy by being able to do `lo < num < hi`.

In [33]:
def solve_2(inp=test2):
    total = 0
    for pol in parse_2(inp):
        if pol.lo <= pol.password.count(pol.letter) <= pol.hi:
            total += 1

    return total

assert solve_2() == 2
printmd(f"Day 2A: **{solve_2(inp2)}**")


Day 2A: **424**

**Part 2:** Now the two nums in the beginning are positions, starting from 1. (no zero index). Only one of those positions can have the given letter. 

In [34]:
def solve_2b(inp=test2):
    total = 0
    for pol in parse_2(inp):
        char1 = pol.password[pol.lo-1]
        char2 = pol.password[pol.hi-1]
        if (pol.letter == char1) != (pol.letter == char2):
            total += 1
    return total

assert solve_2b() == 1
printmd(f"Day 2B: **{solve_2b(inp2)}**")

Day 2B: **747**

## Day 3